In [1]:
import os
import pandas as pd
import pandasql as ps
import numpy as np
from datetime import datetime

#-- Data names
# directory
adnimergefilename="FilesMerged.xlsx"
savedir="/mnt/f/Tirocinio_HDD/Data/ADNI/"
dir_ATN="/mnt/f/Tirocinio_HDD/Data/ADNI/ATN_info/"
dir_adnimerge="/mnt/f/Tirocinio_HDD/Data/ADNI/"+adnimergefilename

In [2]:
adnimerge=pd.read_excel(dir_adnimerge,sheet_name="MAIN")
AV45=pd.read_excel(dir_ATN+"UC Berkeley - AV45 Analysis.xlsx")
subAV45=[]
AV1451=pd.read_excel(dir_ATN+"UC Berkeley - AV1451 Analysis.xlsx")
subAV1451=[]
FBB=pd.read_excel(dir_ATN+"UC Berkeley - FBB Analysis.xlsx")
subFBB=[]
FDG=pd.read_excel(dir_ATN+"UC Berkeley - FDG Analysis .xlsx")
subFDG=[]

In [3]:
'''
### TEST SECTION ####
tmp=AV45[AV45['RID']==4177] #prendi gruppo corrispondente
tmp=tmp.reset_index(drop=True)
print(tmp.loc[0]['EXAMDATE'].date())
dateadni=pd.to_datetime(adnimerge[adnimerge['RID']==4177]['ACQUISITION DATE'].values[0],format='%Y-%m-%d').date() 
print(dateadni)
'''


"\n### TEST SECTION ####\ntmp=AV45[AV45['RID']==4177] #prendi gruppo corrispondente\ntmp=tmp.reset_index(drop=True)\nprint(tmp.loc[0]['EXAMDATE'].date())\ndateadni=pd.to_datetime(adnimerge[adnimerge['RID']==4177]['ACQUISITION DATE'].values[0],format='%Y-%m-%d').date() \nprint(dateadni)\n"

In [4]:
def setRID(df):
    for i in range(0, len(df)):
        df.loc[i, 'RID'] =int( df.loc[i,'PTID'].split('_')[2])


In [5]:
def howmany(df):
    Rids=adnimerge.RID
    df_rids=AV45.RID.unique()
    howmanyes=[]
    count=0
    for rid in Rids:
        if rid in AV45.RID.unique():
            howmanyes.append(rid)
            count+=1
    return howmanyes,count

In [6]:
adnimerge

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286,186.9,14.14,2019-05-31,37,200
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422,231.5,19.86,2017-12-15,2,197
511,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307,176.4,15.17,2018-03-22,30,200
512,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200
513,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927,217.8,19.95,2017-10-17,6,200


In [7]:
def select_from_file(df_file):  #seleziona quelli più vicini alla data presente in adnimerge
    RIDS=adnimerge['RID'].values
    _filteredList=[]
    k=[]
    rids=[]
    tot=0
    nruns=0
    for rid in RIDS:
       
        if not df_file[df_file['RID']==rid].empty: #se il RID esiste sia in adnimerge che nel file in questione, procedi
            nruns+=1
            tmp=df_file[df_file['RID']==rid] #prendi gruppo corrispondente
            tot+=1
            rids.append(rid)
            tmp=tmp.reset_index(drop=True) 
            if len(tmp)>1: #se c'è più di uno filtra data 
                #print(rid)
                #print(tmp)
                low_diff=99999 #differenza minima tra date
                loc_date=-1 #posizione nel file del soggetto con la data più vicina
                dateadni=pd.to_datetime(adnimerge[adnimerge['RID']==rid]['ACQUISITION DATE'].values[0],format='%Y-%m-%d').date() 
                #cicla per ogni data
                for date in range(0,len(tmp)): #date è l'indice in realtà
                    #print(date)
                    date_tmp=tmp.loc[date]['EXAMDATE'].date()
                    datediff=abs(dateadni-date_tmp).days
                    if datediff < low_diff:
                        #nova data più vicina trovata
                        low_diff=datediff
                        loc_date=date #puntatore al nuovo record di data più vicina

                tmp_real=tmp.loc[loc_date]
                #print(tmp.loc[loc_date].index)
                _filteredList.append(tmp_real)
                #upenn3=upenn3[upenn3.columns[upenn3.columns.isin(['RID','EXAMDATE','PTAU','TAU','ABETA'])]]
                #tmp_real=tmp_real[tmp_real.columns[tmp_real.columns.isin(['RID','EXAMDATE','CTX_LH_CAUDALMIDDLEFRONTAL_SUVR','CTX_RH_CAUDALMIDDLEFRONTAL_SUVR','COMPOSITE_SUVR','CTX_LH_ENTORHINAL_SUVR','CTX_RH_ENTORHINAL_SUVR',
                #print(tmp_real)
            else:
                #tmp_real=tmp

                k.append(tmp['RID'].values[0])

           # print("---------------------------------")
            #print(tmp_real)
            #print(adnimerge[adnimerge['RID']==rid])

    #print("Single RIDS:")            
    #print(k)   
    #print("TOTAL:")
   # print(tot)

    #print(AV45_filtered_idxs)
    #print(len(AV45_filtered_idxs))
    #AV45_filtered=AV45[AV45['EXAMDATE'].isin(AV45_filtered_dates) and AV45['RID'].isin(AV45_filtered_idxs)]
    #.isin() per prendere i puntatori corretti


    _filtered=pd.DataFrame(_filteredList)   

    for l in k:
        tmp=df_file[df_file['RID']==l]

        _filtered=pd.concat([_filtered,tmp])

    _filtered=_filtered.reset_index(drop=True) 
    #print("Actual filtered length:")
   # print(len(_filtered))
    print("Filtered "+str(nruns)+" records")
    return _filtered

In [8]:
#select_from_file seleziona i RID esistenti nell'excel principale e filtra già per data
AV45_filtered=select_from_file(AV45)

Filtered 97 records


In [9]:
AV45_filtered

,RID,VISCODE,VISCODE2,EXAMDATE,TP,CEREBELLUMGREYMATTER_SUVR,CEREBELLUMGREYMATTER_VOLUME,WHOLECEREBELLUM_SUVR,WHOLECEREBELLUM_VOLUME,ERODED_SUBCORTICALWM_SUVR,...,RIGHT_PUTAMEN_VOLUME,RIGHT_THALAMUS_PROPER_SUVR,RIGHT_THALAMUS_PROPER_VOLUME,RIGHT_VENTRALDC_SUVR,RIGHT_VENTRALDC_VOLUME,RIGHT_VESSEL_SUVR,RIGHT_VESSEL_VOLUME,WM_HYPOINTENSITIES_SUVR,WM_HYPOINTENSITIES_VOLUME,update_stamp
0,4179,init,m78,2018-01-25,Scan4,1.0747,109082,1.2447,132880,2.6191,...,4446.0,1.8178,7784.0,1.8319,3971.0,1.9820,11.0,1.9873,30440.0,2022-04-28 14:01:13
1,4177,init,m78,2018-01-23,Scan4,1.2584,87510,1.4475,110432,3.0827,...,5986.0,1.7019,7435.0,1.9796,3647.0,1.9652,11.0,2.5518,13016.0,2022-04-28 14:01:13
2,4176,init,m78,2018-01-30,Scan4,1.0725,98836,1.2152,122384,2.2928,...,4885.0,1.6388,7192.0,1.6340,3925.0,1.6168,35.0,1.8545,6155.0,2022-04-28 14:01:13
3,4043,init,m78,2018-04-05,Scan4,0.9321,100840,1.1030,128050,2.0635,...,4499.0,1.3652,7307.0,1.5438,3826.0,1.3308,16.0,1.6278,2535.0,2022-04-28 14:01:13
4,4198,v21,m24,2013-09-26,Scan2,1.0412,96129,1.1790,118354,2.4400,...,3595.0,1.6508,5885.0,1.8457,3210.0,1.8015,8.0,1.9083,1886.0,2022-04-28 14:01:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,4197,init,m72,2017-09-21,Scan4,1.0267,118755,1.1638,148673,2.2844,...,3587.0,1.5515,5992.0,1.6454,3998.0,1.6345,6.0,1.6897,13564.0,2022-04-28 14:01:13
93,150,init,m138,2017-09-05,Scan3,1.0642,102375,1.2052,125654,2.4482,...,3904.0,1.4737,6707.0,1.5593,3807.0,1.6799,2.0,1.8088,8512.0,2022-04-28 14:01:11
94,4071,v41,m48,2015-11-11,BL,0.9759,99671,1.1094,125323,1.9203,...,4386.0,1.3195,8286.0,1.5703,4101.0,1.4001,3.0,1.4343,49967.0,2022-04-28 14:01:13
95,210,init,m138,2018-03-07,BL,0.9994,103234,1.1271,127178,2.2330,...,4787.0,1.3870,6694.0,1.5476,3176.0,1.4123,40.0,1.8618,5087.0,2022-04-28 14:01:11


In [10]:
#AV45 PART
AV45_filtered=AV45_filtered[['RID','EXAMDATE','SUMMARYSUVR_WHOLECEREBNORM','WHOLECEREBELLUM_SUVR','COMPOSITE_SUVR','COMPOSITE_REF_SUVR','RIGHT_AMYGDALA_SUVR','RIGHT_AMYGDALA_VOLUME','LEFT_AMYGDALA_SUVR','LEFT_AMYGDALA_VOLUME','RIGHT_HIPPOCAMPUS_SUVR','RIGHT_HIPPOCAMPUS_VOLUME','LEFT_HIPPOCAMPUS_SUVR','LEFT_HIPPOCAMPUS_VOLUME','CTX_RH_CAUDALMIDDLEFRONTAL_SUVR','CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME','CTX_LH_CAUDALMIDDLEFRONTAL_SUVR','CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME','CTX_RH_ENTORHINAL_SUVR','CTX_RH_ENTORHINAL_VOLUME','CTX_LH_ENTORHINAL_SUVR','CTX_LH_ENTORHINAL_VOLUME']]
#AV45_filtered['AV45_SUVR']=0

AV45_filtered=AV45_filtered.assign(AV45_SUVR=lambda x: x.COMPOSITE_SUVR / x.WHOLECEREBELLUM_SUVR)    #norm???



AV45_filtered['AV45_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']=0
AV45_filtered['AV45_NORM_CTX_LH_ENTORHINAL_SUVR']=0
AV45_filtered['AV45_NORM_LEFT_AMYGDALA_SUVR']=0
AV45_filtered['AV45_NORM_LEFT_HIPPOCAMPUS_SUVR']=0

AV45_filtered['AV45_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR']=0
AV45_filtered['AV45_NORM_CTX_RH_ENTORHINAL_SUVR']=0
AV45_filtered['AV45_NORM_RIGHT_AMYGDALA_SUVR']=0
AV45_filtered['AV45_NORM_RIGHT_HIPPOCAMPUS_SUVR']=0

AV45_filtered=AV45_filtered.assign(AV45_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR=lambda x: x.CTX_LH_CAUDALMIDDLEFRONTAL_SUVR / x.WHOLECEREBELLUM_SUVR) 
AV45_filtered=AV45_filtered.assign(AV45_NORM_CTX_LH_ENTORHINAL_SUVR=lambda x: x.CTX_LH_ENTORHINAL_SUVR / x.WHOLECEREBELLUM_SUVR) 
AV45_filtered=AV45_filtered.assign(AV45_NORM_LEFT_AMYGDALA_SUVR=lambda x: x.LEFT_AMYGDALA_SUVR / x.WHOLECEREBELLUM_SUVR) 
AV45_filtered=AV45_filtered.assign(AV45_NORM_LEFT_HIPPOCAMPUS_SUVR=lambda x: x.LEFT_HIPPOCAMPUS_SUVR / x.WHOLECEREBELLUM_SUVR) 

AV45_filtered=AV45_filtered.assign(AV45_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR=lambda x: x.CTX_RH_CAUDALMIDDLEFRONTAL_SUVR / x.WHOLECEREBELLUM_SUVR) 
AV45_filtered=AV45_filtered.assign(AV45_NORM_CTX_RH_ENTORHINAL_SUVR=lambda x: x.CTX_RH_ENTORHINAL_SUVR / x.WHOLECEREBELLUM_SUVR) 
AV45_filtered=AV45_filtered.assign(AV45_NORM_RIGHT_AMYGDALA_SUVR=lambda x: x.RIGHT_AMYGDALA_SUVR / x.WHOLECEREBELLUM_SUVR) 
AV45_filtered=AV45_filtered.assign(AV45_NORM_RIGHT_HIPPOCAMPUS_SUVR=lambda x: x.RIGHT_HIPPOCAMPUS_SUVR / x.WHOLECEREBELLUM_SUVR)

subAV45=(AV45_filtered.RID.values.tolist())

In [11]:
len(subAV45)


97

In [12]:
AV45_filtered

,RID,EXAMDATE,SUMMARYSUVR_WHOLECEREBNORM,WHOLECEREBELLUM_SUVR,COMPOSITE_SUVR,COMPOSITE_REF_SUVR,RIGHT_AMYGDALA_SUVR,RIGHT_AMYGDALA_VOLUME,LEFT_AMYGDALA_SUVR,LEFT_AMYGDALA_VOLUME,...,CTX_LH_ENTORHINAL_VOLUME,AV45_SUVR,AV45_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,AV45_NORM_CTX_LH_ENTORHINAL_SUVR,AV45_NORM_LEFT_AMYGDALA_SUVR,AV45_NORM_LEFT_HIPPOCAMPUS_SUVR,AV45_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR,AV45_NORM_CTX_RH_ENTORHINAL_SUVR,AV45_NORM_RIGHT_AMYGDALA_SUVR,AV45_NORM_RIGHT_HIPPOCAMPUS_SUVR
0,4179,2018-01-25,1.7337,1.2447,2.1580,1.9482,1.6635,1640.0,1.5906,1493.0,...,1984,1.733751,1.870411,1.149835,1.277898,1.132883,1.936611,1.107014,1.336467,1.144613
1,4177,2018-01-23,1.1618,1.4475,1.6817,2.2536,1.4523,1746.0,1.5309,1520.0,...,2120,1.161796,1.213955,1.012573,1.057617,1.133126,1.118895,1.014508,1.003316,1.085734
2,4176,2018-01-30,1.3843,1.2152,1.6822,1.7457,1.3575,1307.0,1.4216,1194.0,...,2114,1.384299,1.450132,0.956139,1.169849,1.123848,1.414993,0.992758,1.117100,1.044273
3,4043,2018-04-05,0.9541,1.1030,1.0524,1.6063,1.0450,1628.0,1.0736,1520.0,...,2189,0.954125,0.954669,0.907434,0.973345,1.102811,0.958840,0.900181,0.947416,1.105984
4,4198,2013-09-26,1.6457,1.1790,1.9402,1.8095,1.6206,1300.0,1.5551,1014.0,...,1546,1.645632,1.549109,1.053690,1.318999,1.261154,1.603393,1.116115,1.374555,1.364122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,4197,2017-09-21,1.4616,1.1638,1.7010,1.6665,1.2727,1323.0,1.2555,830.0,...,770,1.461591,1.508507,0.714985,1.078794,0.957725,1.648737,0.724695,1.093573,1.105259
93,150,2017-09-05,1.5798,1.2052,1.9039,1.7746,1.3341,1301.0,1.3788,860.0,...,1576,1.579738,1.552439,1.223531,1.144042,0.942914,1.561732,1.069200,1.106953,0.930219
94,4071,2015-11-11,1.2807,1.1094,1.4208,1.5535,1.1070,1702.0,1.1107,1368.0,...,2286,1.280692,1.384442,0.978998,1.001172,0.937173,1.288444,0.971426,0.997837,0.953939
95,210,2018-03-07,1.2951,1.1271,1.4597,1.6421,1.0893,1416.0,1.1651,1519.0,...,1429,1.295094,1.312661,0.993878,1.033715,1.114630,1.326768,1.020939,0.966463,1.172212


In [13]:
AV1451_filtered=select_from_file(AV1451)

Filtered 459 records


In [14]:
AV1451_filtered

,RID,VISCODE,VISCODE2,EXAMDATE,INFERIORCEREBELLUM_SUVR,INFERIORCEREBELLUM_VOLUME,ERODED_SUBCORTICALWM_SUVR,ERODED_SUBCORTICALWM_VOLUME,BRAAK1_SUVR,BRAAK1_VOLUME,...,RIGHT_PUTAMEN_VOLUME,RIGHT_THALAMUS_PROPER_SUVR,RIGHT_THALAMUS_PROPER_VOLUME,RIGHT_VENTRALDC_SUVR,RIGHT_VENTRALDC_VOLUME,RIGHT_VESSEL_SUVR,RIGHT_VESSEL_VOLUME,WM_HYPOINTENSITIES_SUVR,WM_HYPOINTENSITIES_VOLUME,update_stamp
0,6810,y1,m12,2021-01-05,0.9853,71907,1.2010,168467,1.4203,4193,...,3881,1.1711,6936,1.1056,4017,1.3455,29.0,0.8293,2626,2022-04-27 15:31:22
1,6721,bl,bl,2019-06-05,0.9812,76509,1.3308,66683,1.2043,3199,...,4680,1.4193,5574,1.4018,3368,1.5773,57.0,1.0468,20396,2022-04-27 15:31:22
2,6683,bl,bl,2019-03-01,0.9746,61661,1.5290,82117,1.5228,2994,...,3713,1.2303,5301,1.3807,3613,1.6845,26.0,1.1425,2479,2022-04-27 15:31:22
3,6602,bl,bl,2018-11-13,0.9365,76821,1.7380,142406,1.6440,3020,...,5656,1.1228,7087,1.1853,4485,1.5606,2.0,1.0542,7750,2022-04-27 15:31:22
4,6850,bl,bl,2020-02-13,0.9455,75816,1.1897,113887,1.0906,2875,...,4086,1.1687,6836,1.2320,3979,1.4846,5.0,0.8947,3199,2022-04-27 15:31:22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,6227,bl,bl,2018-08-23,0.9876,73602,1.0251,192193,1.1054,3432,...,5220,1.1743,7821,1.2068,4306,1.2245,9.0,0.9393,624,2022-04-27 15:31:21
455,6228,bl,bl,2018-03-14,0.9215,65419,1.0971,97732,0.9626,3371,...,4179,1.3116,5680,1.3293,3399,1.4143,7.0,0.8007,756,2022-04-27 15:31:21
456,6288,bl,bl,2018-04-11,0.9399,56990,1.0573,125133,1.0819,3516,...,4671,1.0779,6843,1.1446,3357,1.4157,9.0,0.7533,4089,2022-04-27 15:31:22
457,5228,y4,m102,2021-11-18,0.9433,67005,1.0530,117592,0.9757,3121,...,4153,1.1863,5782,1.1533,3673,1.3538,9.0,0.7428,1167,2022-04-27 15:31:21


In [15]:
#AV1451 PART
#ref INFERIORCEREBELLUM_SUVR
AV1451_filtered=AV1451_filtered[['RID','EXAMDATE','INFERIORCEREBELLUM_SUVR','BRAAK1_SUVR','BRAAK34_SUVR','BRAAK56_SUVR','CTX_LH_CAUDALMIDDLEFRONTAL_SUVR','CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME','CTX_LH_ENTORHINAL_SUVR','CTX_RH_ENTORHINAL_SUVR','CTX_LH_ENTORHINAL_VOLUME','CTX_RH_CAUDALMIDDLEFRONTAL_SUVR','CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME','CTX_RH_ENTORHINAL_VOLUME','CTX_RH_FRONTALPOLE_SUVR','LEFT_AMYGDALA_SUVR','LEFT_AMYGDALA_VOLUME','LEFT_HIPPOCAMPUS_SUVR','LEFT_HIPPOCAMPUS_VOLUME','RIGHT_AMYGDALA_SUVR','RIGHT_AMYGDALA_VOLUME','RIGHT_HIPPOCAMPUS_SUVR','RIGHT_HIPPOCAMPUS_VOLUME']]
for j in range(0,len(AV1451_filtered)):
    AV1451_filtered.loc[j,'AV1451_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']=0.0
    AV1451_filtered.loc[j,'AV1451_NORM_CTX_LH_ENTORHINAL_SUVR']=0.0
    AV1451_filtered.loc[j,'AV1451_NORM_LEFT_AMYGDALA_SUVR']=0.0
    AV1451_filtered.loc[j,'AV1451_NORM_LEFT_HIPPOCAMPUS_SUVR']=0.0

    AV1451_filtered.loc[j,'COMPOSITE']=0.0

    AV1451_filtered.loc[j,'AV1451_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR']=0.0
    AV1451_filtered.loc[j,'AV1451_NORM_CTX_RH_ENTORHINAL_SUVR']=0.0
    AV1451_filtered.loc[j,'AV1451_NORM_RIGHT_AMYGDALA_SUVR']=0.0
    AV1451_filtered.loc[j,'AV1451_NORM_RIGHT_HIPPOCAMPUS_SUVR']=0.0

#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR=lambda x: x.CTX_LH_CAUDALMIDDLEFRONTAL_SUVR / x.INFERIORCEREBELLUM_SUVR) 
#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_CTX_LH_ENTORHINAL_SUVR=lambda x: x.CTX_LH_ENTORHINAL_SUVR / x.INFERIORCEREBELLUM_SUVR) 
#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_LEFT_AMYGDALA_SUVR=lambda x: x.LEFT_AMYGDALA_SUVR / x.INFERIORCEREBELLUM_SUVR) 
#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_LEFT_HIPPOCAMPUS_SUVR=lambda x: x.LEFT_HIPPOCAMPUS_SUVR / x.INFERIORCEREBELLUM_SUVR) 

#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR=lambda x: x.CTX_RH_CAUDALMIDDLEFRONTAL_SUVR / x.INFERIORCEREBELLUM_SUVR) 
#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_CTX_RH_ENTORHINAL_SUVR=lambda x: x.CTX_RH_ENTORHINAL_SUVR / x.INFERIORCEREBELLUM_SUVR) 
#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_RIGHT_AMYGDALA_SUVR=lambda x: x.RIGHT_AMYGDALA_SUVR / x.INFERIORCEREBELLUM_SUVR) 
#AV1451_filtered=AV1451_filtered.assign(AV1451_NORM_RIGHT_HIPPOCAMPUS_SUVR=lambda x: x.RIGHT_HIPPOCAMPUS_SUVR / x.INFERIORCEREBELLUM_SUVR) 


#subAV1451=(AV1451_filtered.RID.values.tolist())

/home/duetochance/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/duetochance/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [16]:
AV1451_filtered

,RID,EXAMDATE,INFERIORCEREBELLUM_SUVR,BRAAK1_SUVR,BRAAK34_SUVR,BRAAK56_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME,CTX_LH_ENTORHINAL_SUVR,CTX_RH_ENTORHINAL_SUVR,...,RIGHT_HIPPOCAMPUS_VOLUME,AV1451_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,AV1451_NORM_CTX_LH_ENTORHINAL_SUVR,AV1451_NORM_LEFT_AMYGDALA_SUVR,AV1451_NORM_LEFT_HIPPOCAMPUS_SUVR,COMPOSITE,AV1451_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR,AV1451_NORM_CTX_RH_ENTORHINAL_SUVR,AV1451_NORM_RIGHT_AMYGDALA_SUVR,AV1451_NORM_RIGHT_HIPPOCAMPUS_SUVR
0,6810,2021-01-05,0.9853,1.4203,1.3069,1.1832,1.1315,5694,1.3884,1.4852,...,3606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6721,2019-06-05,0.9812,1.2043,1.2204,1.1118,1.0588,6304,1.1964,1.2145,...,2685,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6683,2019-03-01,0.9746,1.5228,1.7826,1.7931,2.3234,3495,1.4289,1.6418,...,3445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6602,2018-11-13,0.9365,1.6440,2.2593,2.4594,2.3354,6755,1.6201,1.6773,...,3357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6850,2020-02-13,0.9455,1.0906,1.0830,0.9770,1.1141,3703,1.1804,1.0391,...,3498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,6227,2018-08-23,0.9876,1.1054,1.0696,1.0079,0.9771,6191,1.1065,1.1044,...,4486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
455,6228,2018-03-14,0.9215,0.9626,1.0548,0.9577,0.9452,6110,0.9664,0.9583,...,3748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456,6288,2018-04-11,0.9399,1.0819,1.0798,1.0142,0.9860,7074,1.0748,1.0882,...,4051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,5228,2021-11-18,0.9433,0.9757,1.0469,1.0031,0.9979,6373,0.9610,0.9938,...,3998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
for i in range(0,len(AV1451_filtered)):
    AV1451_filtered.loc[i,'AV1451_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']=AV1451_filtered.loc[i,'CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']/AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR']
    
    AV1451_filtered.loc[i,'AV1451_NORM_CTX_LH_ENTORHINAL_SUVR']=AV1451_filtered.loc[i,'CTX_LH_ENTORHINAL_SUVR'] /AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR']
    AV1451_filtered.loc[i,'AV1451_NORM_LEFT_AMYGDALA_SUVR']=AV1451_filtered.loc[i,'LEFT_AMYGDALA_SUVR' ]/AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR']
    AV1451_filtered.loc[i,'AV1451_NORM_LEFT_HIPPOCAMPUS_SUVR']=AV1451_filtered.loc[i,'LEFT_HIPPOCAMPUS_SUVR'] /AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR'] 

    AV1451_filtered.loc[i,'AV1451_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR']=AV1451_filtered.loc[i,'CTX_RH_CAUDALMIDDLEFRONTAL_SUVR'] /AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR']
    AV1451_filtered.loc[i,'AV1451_NORM_CTX_RH_ENTORHINAL_SUVR']=AV1451_filtered.loc[i,'CTX_RH_ENTORHINAL_SUVR' ]/AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR']
    AV1451_filtered.loc[i,'AV1451_NORM_RIGHT_AMYGDALA_SUVR']=AV1451_filtered.loc[i,'RIGHT_AMYGDALA_SUVR'  ]/AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR']
    AV1451_filtered.loc[i,'AV1451_NORM_RIGHT_HIPPOCAMPUS_SUVR']=AV1451_filtered.loc[i,'RIGHT_HIPPOCAMPUS_SUVR'] /AV1451_filtered.loc[i,'INFERIORCEREBELLUM_SUVR'] 
    
    
subAV1451=(AV1451_filtered.RID.values.tolist())    
    
    

/home/duetochance/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [18]:
#calcolo composite
for i in range(0,len(AV1451_filtered)):
    composite=(AV1451_filtered.loc[i]['BRAAK1_SUVR']+AV1451_filtered.loc[i]['BRAAK34_SUVR']+AV1451_filtered.loc[i]['BRAAK56_SUVR']+(AV1451_filtered.loc[i]['LEFT_HIPPOCAMPUS_SUVR']+AV1451_filtered.loc[i]['LEFT_HIPPOCAMPUS_SUVR'])/2)/(4)/AV1451_filtered.loc[i]['INFERIORCEREBELLUM_SUVR']
    #print(composite)
    #print(composite)
    #print("__________________________")
    AV1451_filtered.at[i,'COMPOSITE']=composite
    

In [19]:
AV1451_filtered

,RID,EXAMDATE,INFERIORCEREBELLUM_SUVR,BRAAK1_SUVR,BRAAK34_SUVR,BRAAK56_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME,CTX_LH_ENTORHINAL_SUVR,CTX_RH_ENTORHINAL_SUVR,...,RIGHT_HIPPOCAMPUS_VOLUME,AV1451_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,AV1451_NORM_CTX_LH_ENTORHINAL_SUVR,AV1451_NORM_LEFT_AMYGDALA_SUVR,AV1451_NORM_LEFT_HIPPOCAMPUS_SUVR,COMPOSITE,AV1451_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR,AV1451_NORM_CTX_RH_ENTORHINAL_SUVR,AV1451_NORM_RIGHT_AMYGDALA_SUVR,AV1451_NORM_RIGHT_HIPPOCAMPUS_SUVR
0,6810,2021-01-05,0.9853,1.4203,1.3069,1.1832,1.1315,5694,1.3884,1.4852,...,3606,1.148381,1.409114,1.621943,1.351061,1.329950,1.394702,1.507358,1.532630,1.388917
1,6721,2019-06-05,0.9812,1.2043,1.2204,1.1118,1.0588,6304,1.1964,1.2145,...,2685,1.079087,1.219323,1.296168,1.356808,1.240267,1.078679,1.237770,1.352629,1.370872
2,6683,2019-03-01,0.9746,1.5228,1.7826,1.7931,2.3234,3495,1.4289,1.6418,...,3445,2.383952,1.466140,1.295301,1.307511,1.634722,2.899959,1.684589,1.410733,1.313565
3,6602,2018-11-13,0.9365,1.6440,2.2593,2.4594,2.3354,6755,1.6201,1.6773,...,3357,2.493753,1.729952,1.959957,1.588788,2.095729,2.738708,1.791030,1.867165,1.625307
4,6850,2020-02-13,0.9455,1.0906,1.0830,0.9770,1.1141,3703,1.1804,1.0391,...,3498,1.178318,1.248440,1.417345,1.428345,1.190137,1.089159,1.098995,1.326600,1.360127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,6227,2018-08-23,0.9876,1.1054,1.0696,1.0079,0.9771,6191,1.1065,1.1044,...,4486,0.989368,1.120393,1.030579,1.130518,1.088345,0.966484,1.118267,1.074322,1.117659
455,6228,2018-03-14,0.9215,0.9626,1.0548,0.9577,0.9452,6110,0.9664,0.9583,...,3748,1.025719,1.048725,1.305480,1.325122,1.138416,0.999132,1.039935,1.266413,1.363321
456,6288,2018-04-11,0.9399,1.0819,1.0798,1.0142,0.9860,7074,1.0748,1.0882,...,4051,1.049048,1.143526,1.220449,1.185020,1.140999,1.048835,1.157783,1.259602,1.156932
457,5228,2021-11-18,0.9433,0.9757,1.0469,1.0031,0.9979,6373,0.9610,0.9938,...,3998,1.057882,1.018764,1.157956,1.202269,1.102459,1.052687,1.053535,1.169087,1.244249


In [20]:
AV1451_filtered.COMPOSITE

0      1.329950
1      1.240267
2      1.634722
3      2.095729
4      1.190137
         ...   
454    1.088345
455    1.138416
456    1.140999
457    1.102459
458    1.020731
Name: COMPOSITE, Length: 459, dtype: float64

In [21]:
len(subAV1451)

459

In [22]:
FBB_filtered=select_from_file(FBB)   

Filtered 146 records


In [23]:
FBB_filtered

,RID,VISCODE,VISCODE2,EXAMDATE,TP,CEREBELLUMGREYMATTER_SUVR,CEREBELLUMGREYMATTER_VOLUME,WHOLECEREBELLUM_SUVR,WHOLECEREBELLUM_VOLUME,ERODED_SUBCORTICALWM_SUVR,...,RIGHT_PUTAMEN_VOLUME,RIGHT_THALAMUS_PROPER_SUVR,RIGHT_THALAMUS_PROPER_VOLUME,RIGHT_VENTRALDC_SUVR,RIGHT_VENTRALDC_VOLUME,RIGHT_VESSEL_SUVR,RIGHT_VESSEL_VOLUME,WM_HYPOINTENSITIES_SUVR,WM_HYPOINTENSITIES_VOLUME,update_stamp
0,6721,bl,bl,2019-05-16,BL,1.0500,114087,1.2241,136094,2.3231,...,4680,1.4337,5574,1.8024,3368,1.4175,57.0,1.8683,20396,2022-04-28 13:40:16
1,6683,bl,bl,2019-03-05,BL,1.0989,100951,1.2295,120127,2.5639,...,3713,1.5893,5301,1.7477,3613,1.8715,26.0,2.0539,2479,2022-04-28 13:40:16
2,6433,bl,bl,2018-08-23,BL,1.0655,123149,1.1822,141586,2.6875,...,4540,1.8584,5755,1.9303,3692,2.1105,130.0,2.1793,2652,2022-04-28 13:40:16
3,6549,bl,bl,2018-10-09,BL,1.0115,103673,1.1374,127143,2.0583,...,4196,1.6531,7145,1.6900,3969,1.7416,45.0,1.5928,2872,2022-04-28 13:40:16
4,6072,bl,bl,2017-09-14,BL,1.0864,89063,1.2626,113149,2.5892,...,3767,1.7500,5096,1.8269,3253,2.0393,8.0,1.8700,1644,2022-04-28 13:40:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,6161,bl,bl,2018-05-24,BL,1.0451,100659,1.2035,126855,2.2366,...,4975,1.4059,6886,1.6224,3862,1.3822,4.0,1.6216,2969,2022-04-28 13:40:16
142,6148,bl,bl,2018-02-22,BL,1.0369,107214,1.1960,134558,2.4533,...,3463,1.2815,5491,1.7763,3249,1.2580,7.0,2.0020,1302,2022-04-28 13:40:16
143,6335,y2,m24,2020-07-22,BL,0.9426,117561,1.1438,147092,2.3071,...,5244,1.5664,7954,1.8207,4497,1.6180,38.0,1.7285,1361,2022-04-28 13:40:16
144,6118,y2,m24,2020-01-30,BL,1.0615,90903,1.2421,115135,2.1301,...,3860,1.4677,6085,1.7522,3430,1.4217,16.0,1.6307,1519,2022-04-28 13:40:16


In [24]:
#FBB part
#ref SUMMARYSUVR_WHOLECEREBNORM_1.08CUTOFF

In [25]:
FBB_filtered=FBB_filtered[['RID','EXAMDATE','COMPOSITE_REF_SUVR','CTX_LH_CAUDALMIDDLEFRONTAL_SUVR','CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME','CTX_LH_ENTORHINAL_SUVR','CTX_LH_ENTORHINAL_VOLUME','CTX_RH_CAUDALMIDDLEFRONTAL_SUVR','CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME','CTX_RH_ENTORHINAL_SUVR','CTX_RH_ENTORHINAL_VOLUME','LEFT_AMYGDALA_SUVR','LEFT_AMYGDALA_VOLUME','LEFT_HIPPOCAMPUS_SUVR','LEFT_HIPPOCAMPUS_VOLUME','RIGHT_AMYGDALA_SUVR','RIGHT_AMYGDALA_VOLUME','RIGHT_HIPPOCAMPUS_SUVR','RIGHT_HIPPOCAMPUS_VOLUME']]

FBB_filtered['FBB_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']=0.0
FBB_filtered['FBB_NORM_CTX_LH_ENTORHINAL_SUVR']=0.0
FBB_filtered['FBB_NORM_LEFT_AMYGDALA_SUVR']=0.0
FBB_filtered['FBB_NORM_LEFT_HIPPOCAMPUS_SUVR']=0.0

FBB_filtered['COMPOSITE']=0 #è il summarysvur 1.08 cutoff?

FBB_filtered['FBB_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR']=0.0
FBB_filtered['FBB_NORM_CTX_RH_ENTORHINAL_SUVR']=0.0
FBB_filtered['FBB_NORM_RIGHT_AMYGDALA_SUVR']=0.0
FBB_filtered['FBB_NORM_RIGHT_HIPPOCAMPUS_SUVR']=0.0

#FBB_filtered=FBB_filtered.assign(FBB_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR=lambda x: x.CTX_LH_CAUDALMIDDLEFRONTAL_SUVR / x['COMPOSITE_REF_SUVR']) 
#FBB_filtered=FBB_filtered.assign(FBB_NORM_CTX_LH_ENTORHINAL_SUVR=lambda x: x.CTX_LH_ENTORHINAL_SUVR / x['COMPOSITE_REF_SUVR']) 
#FBB_filtered=FBB_filtered.assign(FBB_NORM_LEFT_AMYGDALA_SUVR=lambda x: x.LEFT_AMYGDALA_SUVR / x['COMPOSITE_REF_SUVR']) 
#FBB_filtered=FBB_filtered.assign(FBB_NORM_LEFT_HIPPOCAMPUS_SUVR=lambda x: x.LEFT_HIPPOCAMPUS_SUVR / x['COMPOSITE_REF_SUVR']) 

#FBB_filtered=FBB_filtered.assign(FBB_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR=lambda x: x.CTX_RH_CAUDALMIDDLEFRONTAL_SUVR / x['COMPOSITE_REF_SUVR']) 
#FBB_filtered=FBB_filtered.assign(FBB_NORM_CTX_RH_ENTORHINAL_SUVR=lambda x: x.CTX_RH_ENTORHINAL_SUVR / x['COMPOSITE_REF_SUVR']) 
#FBB_filtered=FBB_filtered.assign(FBB_NORM_RIGHT_AMYGDALA_SUVR=lambda x: x.RIGHT_AMYGDALA_SUVR / x['COMPOSITE_REF_SUVR']) 
#FBB_filtered=FBB_filtered.assign(FBB_NORM_RIGHT_HIPPOCAMPUS_SUVR=lambda x: x.RIGHT_HIPPOCAMPUS_SUVR / x['COMPOSITE_REF_SUVR'])

#FBB_filtered=FBB_filtered.assign(COMPOSITE=lambda x: x['COMPOSITE_REF_SUVR']) 

#subFBB=(FBB_filtered.RID.values.tolist())

/tmp/ipykernel_237/1972567902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBB_filtered['FBB_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']=0.0
/tmp/ipykernel_237/1972567902.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBB_filtered['FBB_NORM_CTX_LH_ENTORHINAL_SUVR']=0.0
/tmp/ipykernel_237/1972567902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [26]:
FBB_filtered

,RID,EXAMDATE,COMPOSITE_REF_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME,CTX_LH_ENTORHINAL_SUVR,CTX_LH_ENTORHINAL_VOLUME,CTX_RH_CAUDALMIDDLEFRONTAL_SUVR,CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME,CTX_RH_ENTORHINAL_SUVR,...,RIGHT_HIPPOCAMPUS_VOLUME,FBB_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,FBB_NORM_CTX_LH_ENTORHINAL_SUVR,FBB_NORM_LEFT_AMYGDALA_SUVR,FBB_NORM_LEFT_HIPPOCAMPUS_SUVR,COMPOSITE,FBB_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR,FBB_NORM_CTX_RH_ENTORHINAL_SUVR,FBB_NORM_RIGHT_AMYGDALA_SUVR,FBB_NORM_RIGHT_HIPPOCAMPUS_SUVR
0,6721,2019-05-16,1.8474,1.2338,6304,1.1866,1805.0,1.2464,5420,1.1123,...,2685,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1,6683,2019-03-05,1.8596,2.2985,3495,1.3210,1673.0,2.3747,2881,1.4190,...,3445,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
2,6433,2018-08-23,1.9259,2.0479,5938,1.6715,1785.0,2.0148,6570,1.5148,...,4208,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
3,6549,2018-10-09,1.6166,1.9286,4067,1.3781,1731.0,1.9379,4955,1.2821,...,4183,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
4,6072,2017-09-14,1.8630,2.2190,4101,1.4867,1689.0,2.1578,3501,1.4697,...,2900,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,6161,2018-05-24,1.7469,1.5234,4991,1.2254,2372.0,1.4786,6005,1.2414,...,4593,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
142,6148,2018-02-22,1.8681,1.2996,4658,1.2806,1116.0,1.2093,4594,1.2136,...,3081,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
143,6335,2020-07-22,1.7838,1.2407,6162,1.0302,1867.0,1.2585,6552,0.9891,...,4459,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
144,6118,2020-01-30,1.7420,1.3294,4456,1.0603,2437.0,1.2669,4250,1.0758,...,3738,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [27]:
for i in range(0,len(FBB_filtered)):
    FBB_filtered.loc[i,'FBB_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']=FBB_filtered.loc[i,'CTX_LH_CAUDALMIDDLEFRONTAL_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'FBB_NORM_CTX_LH_ENTORHINAL_SUVR']=FBB_filtered.loc[i,'CTX_LH_ENTORHINAL_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'FBB_NORM_LEFT_AMYGDALA_SUVR']=FBB_filtered.loc[i,'LEFT_AMYGDALA_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'FBB_NORM_LEFT_HIPPOCAMPUS_SUVR']=FBB_filtered.loc[i,'LEFT_HIPPOCAMPUS_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'FBB_NORM_CTX_RH_ENTORHINAL_SUVR']=FBB_filtered.loc[i,'CTX_RH_ENTORHINAL_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'FBB_NORM_RIGHT_AMYGDALA_SUVR']=FBB_filtered.loc[i,'RIGHT_AMYGDALA_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'FBB_NORM_RIGHT_HIPPOCAMPUS_SUVR']=FBB_filtered.loc[i,'RIGHT_HIPPOCAMPUS_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'FBB_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR']=FBB_filtered.loc[i,'CTX_RH_CAUDALMIDDLEFRONTAL_SUVR']/FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']
    FBB_filtered.loc[i,'COMPOSITE']=FBB_filtered.loc[i,'COMPOSITE_REF_SUVR']




subFBB=(FBB_filtered.RID.values.tolist())

/home/duetochance/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [28]:
len(subFBB)

146

In [29]:
FBB_filtered

,RID,EXAMDATE,COMPOSITE_REF_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME,CTX_LH_ENTORHINAL_SUVR,CTX_LH_ENTORHINAL_VOLUME,CTX_RH_CAUDALMIDDLEFRONTAL_SUVR,CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME,CTX_RH_ENTORHINAL_SUVR,...,RIGHT_HIPPOCAMPUS_VOLUME,FBB_NORM_CTX_LH_CAUDALMIDDLEFRONTAL_SUVR,FBB_NORM_CTX_LH_ENTORHINAL_SUVR,FBB_NORM_LEFT_AMYGDALA_SUVR,FBB_NORM_LEFT_HIPPOCAMPUS_SUVR,COMPOSITE,FBB_NORM_CTX_RH_CAUDALMIDDLEFRONTAL_SUVR,FBB_NORM_CTX_RH_ENTORHINAL_SUVR,FBB_NORM_RIGHT_AMYGDALA_SUVR,FBB_NORM_RIGHT_HIPPOCAMPUS_SUVR
0,6721,2019-05-16,1.8474,1.2338,6304,1.1866,1805.0,1.2464,5420,1.1123,...,2685,0.667858,0.642308,0.636949,0.639710,1.8474,0.674678,0.602089,0.634459,0.610209
1,6683,2019-03-05,1.8596,2.2985,3495,1.3210,1673.0,2.3747,2881,1.4190,...,3445,1.236018,0.710368,0.825823,0.796569,1.8596,1.276995,0.763067,0.784362,0.771510
2,6433,2018-08-23,1.9259,2.0479,5938,1.6715,1785.0,2.0148,6570,1.5148,...,4208,1.063347,0.867906,0.926580,0.971753,1.9259,1.046160,0.786541,0.911885,0.968794
3,6549,2018-10-09,1.6166,1.9286,4067,1.3781,1731.0,1.9379,4955,1.2821,...,4183,1.192998,0.852468,0.970061,0.909625,1.6166,1.198750,0.793084,1.001856,0.929791
4,6072,2017-09-14,1.8630,2.2190,4101,1.4867,1689.0,2.1578,3501,1.4697,...,2900,1.191090,0.798014,0.881374,0.866291,1.8630,1.158239,0.788889,0.918894,0.872410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,6161,2018-05-24,1.7469,1.5234,4991,1.2254,2372.0,1.4786,6005,1.2414,...,4593,0.872059,0.701471,0.646574,0.772912,1.7469,0.846414,0.710630,0.660828,0.801305
142,6148,2018-02-22,1.8681,1.2996,4658,1.2806,1116.0,1.2093,4594,1.2136,...,3081,0.695680,0.685509,0.714095,0.787859,1.8681,0.647342,0.649644,0.670093,0.733580
143,6335,2020-07-22,1.7838,1.2407,6162,1.0302,1867.0,1.2585,6552,0.9891,...,4459,0.695538,0.577531,0.671264,0.752439,1.7838,0.705516,0.554490,0.645420,0.757876
144,6118,2020-01-30,1.7420,1.3294,4456,1.0603,2437.0,1.2669,4250,1.0758,...,3738,0.763146,0.608668,0.729851,0.762629,1.7420,0.727268,0.617566,0.698220,0.783123


In [30]:
#FDG_filtered=select_from_file(FDG) #no, bisonga fare in altro modo per questo

In [31]:
FDG

,COLPROT,RID,VISCODE,VISCODE2,EXAMDATE,UID,ROINAME,MEAN,MAX,STDEV,TOTVOX,update_stamp
0,ADNI1,2,nv,m06,2006-03-08,I26808,metaroi,1.333130,1.644416,0.165853,1142,2022-03-31 15:51:55
1,ADNI1,2,nv,m06,2006-03-08,I26808,pons-vermis,1.090213,1.554459,0.169603,1779,2022-03-31 15:51:55
2,ADNI1,2,nv,bl,2005-09-22,I26812,metaroi,1.251735,1.671892,0.158911,1142,2022-03-31 15:51:55
3,ADNI1,2,nv,bl,2005-09-22,I26812,pons-vermis,0.936819,1.297328,0.107474,1779,2022-03-31 15:51:55
4,ADNI1,3,nv,m06,2006-03-13,I26850,pons-vermis,1.109036,1.517615,0.183215,1779,2022-03-31 15:51:55
...,...,...,...,...,...,...,...,...,...,...,...,...
7393,ADNI3,6850,bl,bl,2020-02-21,I1299320,pons-vermis,1.178196,1.408290,0.108581,1779,2022-03-31 15:51:57
7394,ADNI3,6852,bl,bl,2020-02-26,I1308739,metaroi,1.229679,1.533859,0.126011,1142,2022-03-31 15:51:57
7395,ADNI3,6852,bl,bl,2020-02-26,I1308739,pons-vermis,1.044014,1.324239,0.115930,1779,2022-03-31 15:51:57
7396,ADNI3,6854,bl,bl,2020-02-24,I1301874,metaroi,1.226839,1.487093,0.119725,1142,2022-03-31 15:51:57


In [32]:
#FDG part
#The provided metaROI mean must be 
#intensity normalized by dividing it by the mean of the top 50% of the pons/vermis 
#reference region

In [33]:
FDG_filtered=FDG[FDG['RID'].isin(adnimerge['RID'].values)]
listFDG=[]
listFDG_SKIPPED=[]
for rid in adnimerge['RID'].values:
    fdg=FDG_filtered[FDG_filtered['RID'].isin([rid])]
    #print(fdg)
    #print("-----") 
    #ora cerca data
    fdg=fdg.reset_index(drop=True) 
    #print("BEFORE")
   # print(fdg)
   
    if len(fdg)>3: #se c'è più di uno filtra data 
                #print(rid)
                #print(tmp)
                #print("AFTER")
                #print("ORIG fdg")
                #print(fdg)
                #print("--------------------------------")

                low_diff=99999 #differenza minima tra date
                 #posizione nel file del soggetto con la data più vicina
                dateadni=pd.to_datetime(adnimerge[adnimerge['RID']==rid]['ACQUISITION DATE'].values[0],format='%Y-%m-%d').date() 
                #cicla per ogni data
                best_date_loc=-1
                for elem in range(0,len(fdg)): #per ogni gruppo fdg
                    #print("ROINAME: "+str(fdg.loc[elem]['ROINAME']))
                    
                   
                    #print(fdg.loc[elem])
                    #print(date)
                    
                    date_tmp=fdg.loc[elem]['EXAMDATE'].date() #prendi una data
                    datediff=abs(dateadni-date_tmp).days
                    if datediff < low_diff:
                        #nova data più vicina trovata
                        low_diff=datediff
                        best_date=elem
                        
                        BestD=fdg.loc[elem]['EXAMDATE']
                #fdgi=fdg[fdg['EXAMDATE']==best_date]
                fdgi=fdg[fdg['EXAMDATE']==BestD]
                #print("--------------------------------")
                #print(str(best_date)+" - "+str(low_diff)+" - "+str(BestD))
                
                #print("FDG")
                #print(fdgi)
                #print("---------------")
                meta=fdgi[fdgi['ROINAME']=='metaroi']
                meta=meta.reset_index(drop=True) 
                pv=fdgi[fdgi['ROINAME']=='pons-vermis']
                meta['MEAN_NORM']=meta.MEAN.values/pv.MEAN.values
                #print("META")
                #print(meta)
                listFDG.append(meta.values[0])
                #print(listFDG)
                #print("############")
    elif fdg.empty:
        #not add
        listFDG_SKIPPED.append(rid)
    else:
        #print("AFTER - no doppi")
       # print(fdg)

        #fai direttamente il calcolo
        meta=fdg[fdg['ROINAME']=='metaroi']
        pv=fdg[fdg['ROINAME']=='pons-vermis']
        """print("VALUES")
        print("META------")
        print(meta)
        print("PONS VERMIS---------")
        print(pv)"""
        meta=meta.reset_index(drop=True) 
        meta['MEAN_NORM']=meta.MEAN.values/pv.MEAN.values
        
        """ 
        print("DIVISION #####################")
        print(meta.MEAN.values/pv.MEAN.values)
        print(meta)
        """
        listFDG.append(meta.values[0])
        
        """
        print("## >LIST")
        print(listFDG)
        print("----------")"""

                     

In [34]:
len(listFDG)

364

In [35]:
outFDG=pd.DataFrame(listFDG,columns=['COLPROT', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'UID', 'ROINAME',
       'MEAN', 'MAX', 'STDEV', 'TOTVOX', 'update_stamp','NORM_MEAN'])


In [36]:
subFDG=outFDG.RID
subFDG

0      6810
1      6655
2      6721
3      6683
4      6825
       ... 
359    5234
360    5185
361    5266
362    5203
363    5228
Name: RID, Length: 364, dtype: int64

In [37]:
outFDG[outFDG.RID==6850]

,COLPROT,RID,VISCODE,VISCODE2,EXAMDATE,UID,ROINAME,MEAN,MAX,STDEV,TOTVOX,update_stamp,NORM_MEAN
6,ADNI3,6850,bl,bl,2020-02-21,I1299320,metaroi,1.133334,1.411655,0.141195,1142,2022-03-31 15:51:57,0.961923


In [38]:
def saveexcel(parta,filename):
    writer = pd.ExcelWriter(filename)
    parta.to_excel(writer, sheet_name='Main')
    workbook  = writer.book
    worksheet = writer.sheets['Main']
    worksheet.set_column(1, 37, 37)
    writer.save()

In [39]:
#create the separate excels
datadir="/mnt/f/Tirocinio_HDD/Data/ADNI/"
saveexcel(outFDG,str(datadir+"FDG_Analysis_Filtered.xlsx"))
saveexcel(FBB_filtered,str(datadir+"FBB_Analysis_Filtered.xlsx"))
saveexcel(AV45_filtered,str(datadir+"AV45_Analysis_Filtered.xlsx"))
saveexcel(AV1451_filtered,str(datadir+"AV1451_Analysis_Filtered.xlsx"))



In [40]:
#subFDG
#subFBB
#subAV1451
#subAV45

adnimerge['AV45']=0
adnimerge['AV1451']=0
adnimerge['FBB']=0
adnimerge['FDG']=0
adnimerge

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,0,0,0
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,0,0,0
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286,186.9,14.14,2019-05-31,37,200,0,0,0,0
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,0,0,0
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422,231.5,19.86,2017-12-15,2,197,0,0,0,0
511,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307,176.4,15.17,2018-03-22,30,200,0,0,0,0
512,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200,0,0,0,0
513,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927,217.8,19.95,2017-10-17,6,200,0,0,0,0


In [41]:
adnimerge.at[adnimerge[adnimerge.RID.isin(subFDG)].index,'FDG']=1
adnimerge.at[adnimerge[adnimerge.RID.isin(subFBB)].index,'FBB']=1
adnimerge.at[adnimerge[adnimerge.RID.isin(subAV1451)].index,'AV1451']=1
adnimerge.at[adnimerge[adnimerge.RID.isin(subAV45)].index,'AV45']=1

In [42]:
adnimerge

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,1,0,1
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,1,1,1
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286,186.9,14.14,2019-05-31,37,200,0,1,1,1
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,1,1,1
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422,231.5,19.86,2017-12-15,2,197,0,1,0,1
511,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307,176.4,15.17,2018-03-22,30,200,0,1,0,0
512,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200,0,1,0,0
513,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927,217.8,19.95,2017-10-17,6,200,0,1,0,1


In [43]:
"""
AV45:
CTX_LH_CAUDALMIDDLEFRONTAL_SUVR
CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME
CTX_LH_ENTORHINAL_SUVR
CTX_LH_ENTORHINAL_VOLUME
CTX_RH_CAUDALMIDDLEFRONTAL_SUVR
CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME
CTX_RH_ENTORHINAL_SUVR
CTX_RH_ENTORHINAL_VOLUME
LEFT_AMYGDALA_SUVR
LEFT_AMYGDALA_VOLUME
LEFT_HIPPOCAMPUS_SUVR
LEFT_HIPPOCAMPUS_VOLUME
RIGHT_AMYGDALA_SUVR
RIGHT_AMYGDALA_VOLUME
RIGHT_HIPPOCAMPUS_SUVR
RIGHT_HIPPOCAMPUS_VOLUME
COMPOSITE_SUVR:
A florbetapir cortical summary SUVR can be calculated by dividing “COMPOSITE_SUVR” by WHOLECEREBELLUM_SUVR


AV1451
COMPOSITE: 
composit da creare, dividere somma dei braak per INFERIORCEREBELLUM_SUVR
braak:
BRAAK1_SUVR
BRAAK34_SUVR
BRAAK56_SUVR


CTX_LH_CAUDALMIDDLEFRONTAL_SUVR
CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME
CTX_LH_ENTORHINAL_SUVR
CTX_LH_ENTORHINAL_VOLUME
CTX_RH_CAUDALMIDDLEFRONTAL_SUVR
CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME
CTX_RH_ENTORHINAL_VOLUME
CTX_RH_FRONTALPOLE_SUVR
LEFT_AMYGDALA_SUVR
LEFT_AMYGDALA_VOLUME
LEFT_HIPPOCAMPUS_SUVR
LEFT_HIPPOCAMPUS_VOLUME
RIGHT_AMYGDALA_SUVR
RIGHT_AMYGDALA_VOLUME
RIGHT_HIPPOCAMPUS_SUVR
RIGHT_HIPPOCAMPUS_VOLUME

FBB

CTX_LH_CAUDALMIDDLEFRONTAL_SUVR
CTX_LH_CAUDALMIDDLEFRONTAL_VOLUME
CTX_LH_ENTORHINAL_SUVR
CTX_LH_ENTORHINAL_VOLUME
CTX_RH_CAUDALMIDDLEFRONTAL_SUVR
CTX_RH_CAUDALMIDDLEFRONTAL_VOLUME
CTX_RH_ENTORHINAL_SUVR
CTX_RH_ENTORHINAL_VOLUME
LEFT_AMYGDALA_SUVR
LEFT_AMYGDALA_VOLUME
LEFT_HIPPOCAMPUS_SUVR
LEFT_HIPPOCAMPUS_VOLUME
RIGHT_AMYGDALA_SUVR
RIGHT_AMYGDALA_VOLUME
RIGHT_HIPPOCAMPUS_SUVR
RIGHT_HIPPOCAMPUS_VOLUME

composite da vedere che anche qui non è chiaro il come




FDG
selezionare metaroi
-The provided metaROI mean must be 
intensity normalized by dividing it by the mean of the top 50% of the pons/vermis 
reference region

quindi prendere il metaroi più vicino in data all'adnimerge e poi dividere il suo valore per il valore più alto dei pons/vermis?
boh

"""

"\nAV45:\nCTX_LH_CAUDALMIDDLEFRONTAL_SUVR\nCTX_LH_CAUDALMIDDLEFRONTAL_VOLUME\nCTX_LH_ENTORHINAL_SUVR\nCTX_LH_ENTORHINAL_VOLUME\nCTX_RH_CAUDALMIDDLEFRONTAL_SUVR\nCTX_RH_CAUDALMIDDLEFRONTAL_VOLUME\nCTX_RH_ENTORHINAL_SUVR\nCTX_RH_ENTORHINAL_VOLUME\nLEFT_AMYGDALA_SUVR\nLEFT_AMYGDALA_VOLUME\nLEFT_HIPPOCAMPUS_SUVR\nLEFT_HIPPOCAMPUS_VOLUME\nRIGHT_AMYGDALA_SUVR\nRIGHT_AMYGDALA_VOLUME\nRIGHT_HIPPOCAMPUS_SUVR\nRIGHT_HIPPOCAMPUS_VOLUME\nCOMPOSITE_SUVR:\nA florbetapir cortical summary SUVR can be calculated by dividing “COMPOSITE_SUVR” by WHOLECEREBELLUM_SUVR\n\n\nAV1451\nCOMPOSITE: \ncomposit da creare, dividere somma dei braak per INFERIORCEREBELLUM_SUVR\nbraak:\nBRAAK1_SUVR\nBRAAK34_SUVR\nBRAAK56_SUVR\n\n\nCTX_LH_CAUDALMIDDLEFRONTAL_SUVR\nCTX_LH_CAUDALMIDDLEFRONTAL_VOLUME\nCTX_LH_ENTORHINAL_SUVR\nCTX_LH_ENTORHINAL_VOLUME\nCTX_RH_CAUDALMIDDLEFRONTAL_SUVR\nCTX_RH_CAUDALMIDDLEFRONTAL_VOLUME\nCTX_RH_ENTORHINAL_VOLUME\nCTX_RH_FRONTALPOLE_SUVR\nLEFT_AMYGDALA_SUVR\nLEFT_AMYGDALA_VOLUME\nLEFT_HIPPOCAMP

In [44]:
#filtro per tenere solo quelli entro 2 anni -> 730 giorni
#da usare dopo aver estratto per tutti le altre metriche
#RIFERITO AI PAZIENTI




In [45]:
#######################################
### Filtro pazienti ######
#filtro per differenza data 

#adnimerge=pd.read_excel(dir_adnimerge,sheet_name="MAIN")
Patients=adnimerge[adnimerge['ADNIMERGE'].isin(['AD','EMCI','LMCI'])]
Patients_RIDS=Patients['RID']
removed=0
rids_removed=[] #lista da usare per poi eliminare le righe dei pazienti con dati ottenuti + di 2 anni dopo 


In [46]:
len(Patients)

227

In [47]:
adnimerge

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,1,0,1
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,1,1,1
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286,186.9,14.14,2019-05-31,37,200,0,1,1,1
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,1,1,1
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422,231.5,19.86,2017-12-15,2,197,0,1,0,1
511,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307,176.4,15.17,2018-03-22,30,200,0,1,0,0
512,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200,0,1,0,0
513,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927,217.8,19.95,2017-10-17,6,200,0,1,0,1


In [48]:
for i in range(0,len(adnimerge)): #fai diventare >1700 a 1700
    if adnimerge.loc[i]['ABETA']=='>1700':
        adnimerge.at[i,'ABETA']=1700
adnimerge.ABETA=adnimerge.ABETA.astype(float)

In [49]:
adnimerge_removed=adnimerge.copy()
for RID in Patients_RIDS:
    if adnimerge[adnimerge['RID'] == RID]['DIFF DATE'].values[0]  >= 731:
            
            if adnimerge[adnimerge['RID'] == RID]['ABETA'].values[0]  > 980.0:
                #va bene quindi skippa
                skip=1
            else:
                #print(adniout.loc[line])
                #print("------")
                #print(line)
                rids_removed.append(RID)
                #adnimerge=adnimerge.drop([line]) #togli paziente con differenza > 2 anni
                #adnimerge.reset_index(inplace=True,drop=True)
                removed=removed+1
            
           
            
adnimerge.reset_index(inplace=True,drop=True)
    

In [50]:

len(rids_removed)
#rids_removed

24

In [51]:
adnimerge_removed=adnimerge_removed[~adnimerge_removed['RID'].isin(rids_removed)] #new main in pratica

In [52]:
adnimerge

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,1,0,1
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,1,1,1
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286.0,186.9,14.14,2019-05-31,37,200,0,1,1,1
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,1,1,1
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422.0,231.5,19.86,2017-12-15,2,197,0,1,0,1
511,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307.0,176.4,15.17,2018-03-22,30,200,0,1,0,0
512,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200,0,1,0,0
513,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927.0,217.8,19.95,2017-10-17,6,200,0,1,0,1


In [53]:
adnimerge_removed=adnimerge_removed.reset_index(drop=True)
adnimerge_removed

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,1,0,1
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,1,1,1
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286.0,186.9,14.14,2019-05-31,37,200,0,1,1,1
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,1,1,1
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422.0,231.5,19.86,2017-12-15,2,197,0,1,0,1
487,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307.0,176.4,15.17,2018-03-22,30,200,0,1,0,0
488,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200,0,1,0,0
489,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927.0,217.8,19.95,2017-10-17,6,200,0,1,0,1


In [54]:
# ora salvo il file principale filtrato bene:


In [55]:
#ASEG+VOLUME
dir_asegvolume='/mnt/f/Tirocinio_HDD/Data/ADNI/aseg+volume_tables/'

#altri excel dove sono? manca _all di sicuro

#_all='/mnt/f/Tirocinio_HDD/Data/ADNI/aseg+volume_tables/'
_Third=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_Third.xlsx')
_Second=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_Second.xlsx')
_Fourth=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_Fourth.xlsx')
_Fifth=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_Fifth.xlsx')
_Sixth=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_Sixth.xlsx')


Lorenza=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_Lorenza.xlsx')
Ilaria=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_Ilaria.xlsx')
Fede=pd.read_excel(dir_asegvolume+'freesurfer_features_aseg+volume_fede.xlsx')

In [56]:
_Fourth=_Fourth[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]
_Second=_Second[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]
_Third=_Third[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]
Lorenza=Lorenza[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]
Ilaria=Ilaria[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]
Fede=Fede[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]
_Sixth=_Sixth[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]
_Fifth=_Fifth[['PTID','Right-Amygdala','Left-Amygdala','lh-caudalmiddlefrontal','rh-caudalmiddlefrontal','lh-entorhinal','rh-entorhinal','Left-Hippocampus','Right-Hippocampus','EstimatedTotalIntraCranialVol']]


_Third['RID']=0
_Fourth['RID']=0
_Second['RID']=0
Lorenza['RID']=0
Fede['RID']=0
Ilaria['RID']=0
_Fifth['RID']=0
_Sixth['RID']=0

setRID(_Second)
setRID(_Fourth)
setRID(_Third)
setRID(Fede)
setRID(Lorenza)
setRID(Ilaria)
setRID(_Sixth)
setRID(_Fifth)

#prendo 
_Second=_Second[_Second['RID'].isin(adnimerge.RID)]
_Third=_Third[_Third['RID'].isin(adnimerge.RID)]
_Fourth=_Fourth[_Fourth['RID'].isin(adnimerge.RID)]
Lorenza=Lorenza[Lorenza['RID'].isin(adnimerge.RID)]
Ilaria=Ilaria[Ilaria['RID'].isin(adnimerge.RID)]
Fede=Fede[Fede['RID'].isin(adnimerge.RID)]
_Sixth=_Sixth[_Sixth['RID'].isin(adnimerge.RID)]
_Fifth=_Fifth[_Fifth['RID'].isin(adnimerge.RID)]

In [57]:
len(_Fourth)+len(_Third)+len(_Second)+len(Lorenza)+len(Ilaria)+len(Fede)+len(_Fifth)+len(_Sixth)
print(len(_Fourth)+len(_Third)+len(_Second)+len(Lorenza)+len(Ilaria)+len(Fede)+len(_Fifth)+len(_Sixth))

total_volumes=pd.concat([Ilaria,Lorenza,Fede,_Fourth,_Third,_Second,_Fifth,_Sixth],ignore_index = True)

514


In [58]:
len(adnimerge_removed)

491

In [59]:
len(total_volumes)

514

In [60]:
total_volumes.to_excel(savedir+'TOTAL_VOLUMES.xlsx',sheet_name='Total')

In [61]:
#adnimerge_new[adnimerge_new['RID'].isin(total_volumes.RID)]

In [62]:
adnimerge_new=adnimerge_removed.copy()
adnimerge_new['VOLUMES']=0
adnimerge_new.at[adnimerge_new[adnimerge_new['RID'].isin(total_volumes.RID)].index,'VOLUMES']=1
adnimerge_new

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG,VOLUMES
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,1,0,1,1
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,1,1,1,1
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286.0,186.9,14.14,2019-05-31,37,200,0,1,1,1,1
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,1,1,1,1
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422.0,231.5,19.86,2017-12-15,2,197,0,1,0,1,1
487,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307.0,176.4,15.17,2018-03-22,30,200,0,1,0,0,1
488,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200,0,1,0,0,1
489,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927.0,217.8,19.95,2017-10-17,6,200,0,1,0,1,1


In [63]:
adnimerge['VOLUMES']=0
adnimerge.at[adnimerge[adnimerge['RID'].isin(total_volumes.RID)].index,'VOLUMES']=1
adnimerge

,Unnamed: 0,index,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG,VOLUMES
0,0,787,305_S_6810,6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,1,0,1,1
1,1,782,098_S_6655,6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,1,1,1,1
2,2,515,126_S_6721,6721,AD,2019-04-24,78.0,M,1286.0,186.9,14.14,2019-05-31,37,200,0,1,1,1,1
3,3,514,126_S_6683,6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,1,1,1,1
4,4,513,123_S_6825,6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,510,99,020_S_5203,5203,SMC,2017-12-13,71.0,F,1422.0,231.5,19.86,2017-12-15,2,197,0,1,0,1,1
511,511,362,129_S_6228,6228,SMC,2018-02-20,75.1,F,1307.0,176.4,15.17,2018-03-22,30,200,0,1,0,0,1
512,512,364,129_S_6288,6288,SMC,2018-04-02,72.6,M,616.3,400.4,39.71,2018-04-17,15,200,0,1,0,0,1
513,513,338,127_S_5228,5228,SMC,2017-10-23,82.2,F,1927.0,217.8,19.95,2017-10-17,6,200,0,1,0,1,1


In [64]:
#SAVE EXCEL
#outFDG
#FBB_filtered
#AV45_filtered
#AV1451_filtered

In [65]:
adnimerge=adnimerge.drop(['Unnamed: 0'],axis=1)
adnimerge_new= adnimerge_new.drop(['Unnamed: 0'],axis=1)

In [66]:
adskipped = pd.read_excel(savedir+adnimergefilename, sheet_name='SUBJECTS_SKIPPED')
adclass=pd.read_excel(savedir+adnimergefilename, sheet_name='CLASSES')


#dataframe of skipped subjects
df=pd.DataFrame(rids_removed,columns=['RID'])
df=df.join(adnimerge[~adnimerge['RID'].isin(rids_removed)],rsuffix='R',lsuffix='L') 
df=df.drop(['RIDR'],axis=1)
df.rename(columns = {'RIDL':'RID'}, inplace = True)
df.index=df.index.astype(int)

with pd.ExcelWriter(savedir+"MERGED_FILES_FINAL.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    adnimerge_new.to_excel(writer, sheet_name="MAIN-Filtered", index=False)
    adnimerge.to_excel(writer, sheet_name="MAIN-Total", index=False)
    adclass.to_excel(writer,sheet_name="CLASSES",index=False)
    adskipped.to_excel(writer, sheet_name="SUBJECTS_SKIPPED_FIRST_PASS", index=False)
    total_volumes.to_excel(writer,sheet_name="VOLUMES",index=False)
    pd.DataFrame(rids_removed,columns=['RID']).to_excel(writer,sheet_name="SUBJS >2Y",index=False)
    #pd.DataFrame(rids_removed,columns=['RID']).to_excel(writer,sheet_name="PATIENTS NOT IN RANGE-from 1st main",index=False)

In [67]:
df

,RID,index,ID,ADNIMERGE,ACQUISITION DATE,AGE,SEX,ABETA,TAU,PTAU,EXAMDATE,DIFF DATE,fMRI Volumes,AV45,AV1451,FBB,FDG,VOLUMES
0,4659,787,305_S_6810,AD,2019-11-02,73.2,M,562.6,299.7,30.43,2019-12-03,31,197,0,1,0,1,1
1,4744,782,098_S_6655,AD,2019-01-07,80.9,M,688.9,394.0,40.35,2019-02-20,44,200,0,1,1,1,1
2,4127,515,126_S_6721,AD,2019-04-24,78.0,M,1286.0,186.9,14.14,2019-05-31,37,200,0,1,1,1,1
3,4868,514,126_S_6683,AD,2019-02-08,66.8,M,866.2,335.9,33.43,2019-03-22,42,200,0,1,1,1,1
4,4891,513,123_S_6825,AD,2019-09-24,73.3,F,599.9,319.6,31.80,2019-10-10,16,197,0,1,1,1,1
5,2130,510,032_S_6602,AD,2018-10-11,63.4,M,495.4,272.3,26.38,2018-11-13,33,197,0,1,0,1,1
6,4351,505,305_S_6850,AD,2020-01-29,79.8,M,1061.0,408.7,30.82,2020-02-24,26,197,0,1,0,1,1
7,4185,502,168_S_6843,AD,2020-01-14,68.3,F,838.8,671.2,69.17,2020-06-08,146,197,0,1,0,1,1
8,4974,491,100_S_6713,AD,2019-04-18,71.2,F,422.0,393.0,41.88,2019-05-02,14,197,0,1,1,1,1
9,2380,58,011_S_6303,AD,2018-04-12,70.4,M,689.7,319.5,32.74,2018-04-26,14,197,0,1,0,1,1
